In [12]:
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
import pandas as pd
import os
import sys
import copy
sys.path.append(r"C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound")
from ecosound.core.measurement import Measurement
from ecosound.core.audiotools import Sound
from ecosound.core.spectrogram import Spectrogram
from ecosound.visualization.grapher_builder import GrapherFactory
pn.extension()

# load dataset
data_file=r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\results\dataset_FS-NN.nc'
dataset = Measurement()
dataset.from_netcdf(data_file)
data_all = dataset.data
data = copy.deepcopy(data_all)
#data = data.sample(n = 10000)

# Widgets
inputs = list(data.columns)
X_variable  = pn.widgets.Select(name='X axis',
                                value='frequency_min',
                                options=inputs)
Y_variable  = pn.widgets.Select(name='Y axis',
                                value='frequency_max',
                                options=inputs)
Color  = pn.widgets.Select(name='Color',
                           value='label_class',
                           options=inputs)
alpha_slider = pn.widgets.FloatSlider(name='Point transparency',
                                      start=0, end=1,
                                      step=0.1,
                                      value=0.2,
                                     )
size_slider = pn.widgets.IntSlider(name='Point size',
                                   start=1,
                                   end=40,
                                   step=1,
                                   value=6,
                                  )

sampling_slider = pn.widgets.FloatSlider(name='Sampling ratio',
                                      start=0, end=1,
                                      step=0.1,
                                      value=0.2,
                                     )
# Stream
tap = hv.streams.Selection1D(index=[data.index.min()])

# Subsample
#@pn.depends(sampling_slider)
def subsample(data1, sampling_slider):
    n_points = int(len(data1)*sampling_slider)
    return data1.sample(n = n_points)

# Scatter plot
@pn.depends(X_variable, Y_variable, Color, alpha_slider, size_slider,sampling_slider)
def scatterplot(X_variable, Y_variable, Color, alpha_slider, size_slider,sampling_slider):
    global data
    data = subsample(data_all, sampling_slider)
    scatter_plot = data.hvplot.scatter(x=X_variable,
                                       y=Y_variable,
                                       c=Color,
                                       s=size_slider,
                                       alpha=alpha_slider,
                                       title= str(len(data)) + '/' + str(len(data_all)) + ' points',
                                       #datashade=True,
                                       tools=['tap'],
                                       active_tools=['wheel_zoom'],
                                      )
    tap.source = scatter_plot   
    #scatter_plot.opts(click_policy="hide")
    return scatter_plot   

# Measurement table
@pn.depends(tap.param.index)
def table(index):  
    if index:
        return pn.pane.DataFrame(data.iloc[index[0]])

# Spectrogram
@pn.depends(tap.param.index)
def spectrogram_plot(index):
    #data.iloc[index[0]]
    if index:
        frame = 0.0625 #3000
        nfft = 0.0853 # 4096
        step = 0.01 # 5
        fmin = 0
        fmax = 1000
        window_type = 'hann'
        time_buffer = 1
        palet = 'jet' # 'binary'

        data_selection = data.iloc[index[0]]

        wavfilename = os.path.join(data_selection.audio_file_dir, data_selection.audio_file_name + data_selection.audio_file_extension)
        t1 = data_selection.time_min_offset - time_buffer
        t2 = data_selection.time_max_offset + time_buffer
        # load audio data
        sound = Sound(wavfilename)
        sound.read(channel=0, chunk=[t1, t2], unit='sec', detrend=True)
        # Calculates  spectrogram
        spectro = Spectrogram(frame, window_type, nfft, step, sound.waveform_sampling_frequency, unit='sec')
        spectro.compute(sound, dB=True, use_dask=False, dask_chunks=40)

        # Plot
        graph = GrapherFactory('SoundPlotter', title=str(index[0]) + ': ' +data_selection.label_class + ' - ' +data_selection.label_subclass, frequency_max=fmax)
        graph.add_data(spectro)
        #graph.colormap = 'binary'
        graph.colormap = palet
        fig, ax = graph.show(display=False)
        return pn.pane.Matplotlib(fig)

# Dashboard
#gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)
tabs = pn.Tabs(('Axes',pn.Column(X_variable, Y_variable, Color, sampling_slider)))
tabs.append(('Style',pn.Column(alpha_slider, size_slider)))
widgets = tabs
tabs2 = pn.Tabs(('Spectrogram',spectrogram_plot))
tabs2.append(('Measurements',table))

dashboard = pn.Column(pn.Row(widgets, scatterplot),tabs2)

dashboard.show()
#dashboard


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


Launching server at http://localhost:59875
